### **Objective**

This analysis is a continuation a prior study carried out in H1 2023 to test the Friday effect hypothesis. Initiated by a perceived lull in commercial activity on Fridays across London post-COVID, the following analysis seeks to uncover what the data says about H2 2023, identify opportunities for change, and in subsequent months, measure the effect of solutions deployed to generate more activity on Fridays. It seeks to provide a basis for discussions along the lines of
- Is there infact less activity on Fridays compared to other days of the week?
- If so, what are some contributing factors to this decline?
- Is this trend sustained over the full year?
- Is this trend more noticable in terms of footfall traffic, commercial activity or both?
- How much of an impact could this have on business growth across London?

***Keywords***

- CAZ - Central Activities Zone
- Hex - Spatial hexagons
- MSOA - Middle Layer Super Output Area
- LSOA - Lower Layer Super Output Area
- BID - Business Improvement Districts
- HS - High Streets
- DOW - Days of the Week

### **Set Up The Environment**

In [ ]:
import numpy as np
import pandas as pd
import ipywidgets
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from sklearn import linear_model
import geopandas as gpd
import contextily as ctx
import numpy as np
import matplotlib.pylab as pl
from matplotlib.colors import ListedColormap
from scipy.spatial.distance import cdist
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
from itertools import chain

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#initialize spark context
spark = SparkSession.builder.master("local[*]").appName('Friday Effect Analysis').getOrCreate()

In [ ]:
colours = ['rgb(xx, xx, xx)', 'rgb(xx, xx, xx)', 'rgb(xx, xx, xx)', 'rgb(xx, xx, xx)',
           'rgb(xx, xx, xx)', 'rgb(xx, xx, xx)', 'rgb(xx, xx, xx)', 'rgb(xx, xx, xx)',
           'rgb(xx, xx, xx)', 'rgb(xx, xx, xx)']

In [ ]:
# define visualization functions
def dropdown_plot(which_fig, data: pd.DataFrame, x_col: str, menu_options_list: list, x_axis_is_date = False, plot_type = 'bar', dropdown_type = 'dropdown'):
  """Plots multiple charts with common x-axis and y-axis, using one dropdown menu
     Params
     which_fig: fig, points to an initialised Plotly figure
     data: pd.DataFrame, dataframe to plot figures from
     x_col: str, column to use as x-axis
     x_axis_is_date: boolean, if x_col is a date column
     menu_options_list: list, containing the dropdown menu options
     plot_type: str, type of plot to use (bar or line)
     dropdown_type: str, type of dropdown menu to use (dropdown or button)
  """
    if x_axis_is_date == True:
        data[x_col] = pd.to_datetime(data.index)

    buttons = []

    # button with one option for each dataframe
    for col in menu_options_list:
        buttons.append(dict(method='update',
                            label=col,
                            visible=True,
                            args=[{'y':[data[col]],
                                'x':[data[x_col]],
                                'type': plot_type}, [0]]
                            )
                    )

    # some adjustments to the updatemenus
    updatemenu = []
    your_menu = dict()
    updatemenu.append(your_menu)

    updatemenu[0]['buttons'] = buttons
    updatemenu[0]['direction'] = 'down'
    updatemenu[0]['showactive'] = True
    updatemenu[0]['type'] = dropdown_type
    updatemenu[0]['yanchor'] = 'top'

    which_fig.update_layout(showlegend=True, updatemenus=updatemenu)

def multi_ddplot(which_fig, df_list: list, x_col: str, menu_options_col: str, button_list: list, buttonposition: list, labelposition: list, buttonheight: float, x_is_date = True):
    """Plots multiple charts with common x-axis and y-axis, using multiple dropdown menus
    Params
    which_fig: fig, point to an initialised Plotly figure
    df_list: list, list of dataframes to plot figures from
    x_col: str, column to use as x-axis
    x_is_date: boolean, if x_col is a date column
    menu_options_col: str, column containing the dropdown menu options
    button_list: list, names for dropdown buttons (must be numerical columns in the df)
    buttonposition: list, positions of dropdown buttons on the x-axis
    labelposition: list, positions of labels on the x-axis
    buttonheight: float, position of dropdown menu on the y-axis
    """

    for df in df_list:
        df.sort_values(by=x_col, ascending=True, inplace = True)
        if x_is_date == True:
            df[x_col] = pd.to_datetime(df[x_col])

    # updating each button with arguments to visualise when selected
    button_dict = dict()
    annotation  = []
    updatemenu  = []
    for i, button in enumerate(button_list):
        button_dict[button] = []
        for cat in df_list[0][menu_options_col].unique():
            x = []
            y = []
            traces = []
            for j, df in enumerate(df_list):
                x.append(df[df[menu_options_col] == cat][x_col])
                y.append(df[df[menu_options_col] == cat][button])
                traces.append(j)
            button_dict[button].append(dict(method  = 'restyle',
                                            label   = cat,
                                            visible = True,
                                            args    = [{'y':    y,
                                                        'x':    x,
                                                        'type': 'scatter',
                                                        'visible': True },
                                                       traces
                                                       ]
                                            )
                                        )

        updatemenu.append(dict(buttons = button_dict[button],
                               direction = 'down',
                               showactive = True,
                               type = 'dropdown',
                               x = buttonposition[i],
                               xanchor = "left",
                               y = buttonheight,
                               yanchor = 'top'
                               )
                            )

        annotation.append(dict(text = button_list[i],
                               x = labelposition[i],
                               xref = "paper",
                               y = buttonheight - 0.03,
                               yref = "paper",
                               align = "left",
                               showarrow = False
                               )
                            )

    # adding dropdown menus & menu annotations to the figure
    which_fig.update_layout(showlegend = True, updatemenus = updatemenu)
    which_fig.update_layout(annotations = annotation)

def rgb_to_rgba(rgb_value, alpha):
    """
    Adds the alpha channel to an RGB Value and returns it as an RGBA Value
    Params
    rgb_value: Input RGB Value
    alpha: Alpha Value to add  in range [0,1]

    Return
    RGBA Value
    """
    return f"rgba{rgb_value[3:-1]}, {alpha})"

def plot_signal(which_fig, x_signal, mean_signal, sem_signal, colour, label):
    """
    Plots a mean +- SEM signal using Plotly
    Params
    which_fig: point to an initialised Plotly figure
    x_signal: independent var
    mean_signal: dependent var MEAN
    sem_signal: dependent var SEM
    colour: signal colour (same for line and fill)
    label: hoverinfo label text
    """
    fill_colour = rgb_to_rgba(colour, .2)
    line_colour = rgb_to_rgba(colour, 1)
    which_fig.add_trace(go.Scatter(x=x_signal,
                                   y=mean_signal+sem_signal,
                                   mode='lines',
                                   line=dict(color='rgba(0,0,0,0)'),
                                   showlegend=False,
                                   hoverinfo='skip'
                                   )
                        )
    which_fig.add_trace(go.Scatter(x=x_signal,
                                   y=mean_signal-sem_signal,
                                   mode='lines',
                                   fill='tonexty',
                                   fillcolor=fill_colour,
                                   line=dict(color='rgba(0,0,0,0)'),
                                   showlegend=False,
                                   hoverinfo='skip'
                                   )
                        )
    which_fig.add_trace(go.Scatter(x=x_signal,
                                   y=mean_signal,
                                   mode='lines',
                                   line=dict(color=line_colour),
                                   name=label
                                   )
                        )

def plot_annotated_line(which_fig, x0, x1, y0, y1, colour, dash_type, text, textloc):
    which_fig.add_shape(type='line',
                        x0=x0,
                        y0=y0,
                        x1=x1,
                        y1=y1,
                        line=dict(color=colour,dash=dash_type),
                        xref='x',
                        yref='y'
                        )
    which_fig.add_trace(go.Scatter(x=[x0],
                                   y=[y1],
                                   text=[text],
                                   textposition=textloc,
                                   showlegend=False,
                                   mode="text",
                                   hoverinfo=None,
                                   textfont=dict(color=colour,
                                                 size=14,
                                                 family="Arial",
                                                )
                                    )
                        )

def pretty_plot(which_fig, title, xlabel, ylabel):
    which_fig.update_layout(title = dict(text=title,font = (dict(size=22))))

    # which_fig.update_xaxes(showline=True,zeroline=True,zerolinewidth=2,zerolinecolor="rgba(200,200,200,.5)",gridcolor="rgba(200,200,200,.3)")
    which_fig.update_xaxes(title_text=xlabel, title_font_size=15)
    which_fig.update_yaxes(title_text=ylabel, title_font_size=15, tickfont_size=12,showline=True,zeroline=True,zerolinewidth=2,zerolinecolor="rgba(200,200,200,.5)",gridcolor="rgba(200,200,200,.3)")

    which_fig.update_layout(
        font_family="Arial",
        font_color="rgb(100,100,100)",
    )

def size_plot(which_fig, height, width):
    which_fig.update_layout(
                  autosize=False,
                  width=width,
                  height=height,
                  paper_bgcolor="rgba(0,0,0,0)",
                  plot_bgcolor="rgba(0,0,0,0)",
                  font_color="rgb(255,255,255)")

### **Pre-Processing**

#### Lookup Files

In [ ]:
# Import lookup files
caz_quad_lookup = spark.read.option("header", "true").csv('Q:/Projects/2019-20/Covid-19 Busyness/data/mastercard/CAZ_quad_lookup.csv')
caz_msoa_lookup = spark.read.option("header", "true").csv('Q:/Projects/2019-20/Covid-19 Busyness/data/reference_data/msoa_caz_noncaz_lookup.csv')
caz_hex_lookup = spark.read.option("header", "true").csv('Q:/Projects/2019-20/Covid-19 Busyness/data/reference_data\Hex350_grid_HS_TC_BID_CAZ_lookup.csv')
ref_hex_lookup = spark.read.option("header", "true").csv('Q:/Projects/2019-20/Covid-19 Busyness/data/reference_data\Hex350_grid_Mastercard_Zoom18_ldn_ref_lookup.csv')
quad_hex_lookup = spark.read.option("header", "true").csv('Q:/Projects/2019-20/Covid-19 Busyness/data/reference_data/Hex350_grid_Mastercard_Zoom18_lookup.csv')
hex_grid_lsoa_lookup = spark.read.option("header", "true").csv('Q:/Projects/2019-20/Covid-19 Busyness/data/reference_data/Hex350_grid_lsoa_lookup.csv')
HS_TC_BID_hex_lookup = spark.read.option("header", "true").csv('Q:/Projects/2019-20/Covid-19 Busyness/data/reference_data/Hex350_grid_HS_TC_BID_Bespoke_lookup.csv')
print("caz_quad_lookup shape is {}".format((caz_quad_lookup.count(), len(caz_quad_lookup.columns))))
print("caz_msoa_lookup shape is {}".format((caz_msoa_lookup.count(), len(caz_msoa_lookup.columns))))
print("caz_hex_lookup shape is {}".format((caz_hex_lookup.count(), len(caz_hex_lookup.columns))))
print("ref_hex_lookup shape is {}".format((ref_hex_lookup.count(), len(ref_hex_lookup.columns))))
print("quad_hex_lookup shape is {}".format((quad_hex_lookup.count(), len(quad_hex_lookup.columns))))
print("hex_grid_lsoa_lookup shape is {}".format((hex_grid_lsoa_lookup.count(), len(hex_grid_lsoa_lookup.columns))))
print("HS_TC_BID_hex_lookup shape is {}".format((HS_TC_BID_hex_lookup.count(), len(HS_TC_BID_hex_lookup.columns))))

In [ ]:
# cleanup the lookup files
caz_msoa_lookup = caz_msoa_lookup.withColumnRenamed('msoa', 'msoa_id')
caz_hex_lookup = caz_hex_lookup.withColumnRenamed('Hex_ID', 'hex_id')\
                               .withColumnRenamed( 'CAZ', 'area')
quad_hex_lookup = quad_hex_lookup.withColumnRenamed('Hex_ID', 'hex_id')
ref_hex_lookup = ref_hex_lookup.withColumn('quad_id', ref_hex_lookup['quad_id'].cast(T.LongType()))\
                               .withColumn('hex_id', ref_hex_lookup['hex_id'].cast(T.LongType()))
caz_hex_lookup = caz_hex_lookup.na.fill(value = 'Non-CAZ', subset = 'area')
caz_hex_lookup = caz_hex_lookup.withColumn('hex_id', caz_hex_lookup['hex_id'].cast(T.LongType()))

In [ ]:
# Read in shapefiles
# bt_gdf = gpd.read_file("Q:/Projects/2019-20/Covid-19 Busyness/data/reference_data/shapefiles/hex350_grid_GLA.shp")
# mc_gdf = gpd.read_file("Q:/Projects/2019-20/Covid-19 Busyness/Python/Mastercard 3hr processing/quad_shapefile.shp")

#### Mastercard Transactions Data

In [ ]:
spend_2022 = spark.read.csv("Q:\Projects/2019-20\Covid-19 Busyness\data\mastercard\Processed\MRLI_3yr_compressed\MRLI_3yr_compressed_2022.csv", header=True, inferSchema=True, sep=',')
spend_2023 = spark.read.csv("Q:\Projects/2019-20\Covid-19 Busyness\data\mastercard\Processed\MRLI_3yr_compressed\MRLI_3yr_compressed_2023.csv", header=True, inferSchema=True, sep=',')
spend = spend_2022.union(spend_2023)
spend.count(), len(spend.columns)

In [ ]:
spend.show()

In [ ]:
# # join mastercard spend data to corr hexes and highstreets
spend = spend.withColumn('quad_id', spend['quad_id'].cast(T.LongType()))
spend = spend.join(quad_hex_lookup.select('quad_id', 'hex_id'), on = 'quad_id', how = 'left')
spend = spend.join(caz_hex_lookup.select('hex_id', 'highstreet_name', 'area'), on = 'hex_id', how = 'left')

# extract day, week & month columns from count_date
spend = spend.withColumn('txn_week', F.trunc('count_date', 'week'))\
             .withColumn('txn_month', F.trunc('count_date', 'month'))\
             .withColumn("daynumber", F.dayofweek(F.col('count_date')))
day_labels = {'1': 'Sun', '2': 'Mon', '3': 'Tue', '4': 'Wed', '5': 'Thu', '6': 'Fri', '7': 'Sat'}
mapping_expr = F.create_map([F.lit(x) for x in chain(*day_labels.items())])
spend = spend.withColumn('day', mapping_expr[spend['daynumber']])

#### BT Footfall Data

In [ ]:
# read in & combine trihourly footfall for 2022 & 2023
footfall_2023 = spark.read.csv("Q:\Projects/2019-20\Covid-19 Busyness\data\BT\Processed\hex_grid\hex_3hourly_counts_2023.csv", header=True, inferSchema= True)
footfall_2022 = spark.read.csv("Q:\Projects/2019-20\Covid-19 Busyness\data\BT\Processed\hex_grid\hex_3hourly_counts_2022.csv", header=True, inferSchema= True)
footfall = footfall_2022.union(footfall_2023)
print('2023 footfall data size: ', footfall_2023.count(), len(footfall_2023.columns))
print('2022 footfall data size: ', footfall_2022.count(), len(footfall_2022.columns))
print('Combined footfall data size: ', footfall.count(), len(footfall.columns))

In [ ]:
#add a daynumber column to sort with
footfall = footfall.withColumn("daynumber", F.dayofweek(F.col('count_date')))\
                    .withColumn('week', F.weekofyear(F.col("count_date")))\
                    .withColumn('yr', F.year(F.col("count_date")))\
                    .withColumn("count_week", F.trunc('count_date', 'week'))\
                    .withColumn("count_month", F.trunc('count_date', 'month'))

footfall = footfall.join(caz_hex_lookup.select('hex_id','area','highstreet_name'), on = 'hex_id', how='left')
footfall.show(5)

### **Footfall Analysis**

In [ ]:
#creating an sql view of the hex footfall data
footfall.createOrReplaceTempView('hex_footfall_View')

#### Daily Footfall

In [ ]:
# caz_msoa_lookup = caz_msoa_lookup.toPandas()
feb20_ff = pd.read_csv('Q:/Projects/2019-20/Covid-19 Busyness/data/O2/msoa_hourly_counts_2020-02-01_2020-02-14.csv')
feb20_ff = feb20_ff.merge(caz_msoa_lookup, left_on = 'msoa11cd', right_on = 'msoa_id', how = 'left')
feb20_ff = feb20_ff[['count_date', 'count_type', 'area', 'h9','h10','h11','h12','h13','h14','h15','h16','h17','h18']]\
                    .groupby(by = ['count_date', 'count_type', 'area']).sum(numeric_only = True).reset_index() # summing up footfall in each msoa
feb20_ff['count_date'] = pd.to_datetime(feb20_ff['count_date'])
feb20_ff['count_week'] = feb20_ff['count_date'].dt.to_period('W').apply(lambda r: r.start_time)
feb20_ff['count_month'] = 'Feb 2020'
feb20_ff['day'] = feb20_ff['count_date'].dt.isocalendar().day
days = {1:'Mon', 2:'Tue', 3:'Wed', 4:'Thu', 5:'Fri', 6:'Sat', 7:'Sun'}
feb20_ff['day'] = feb20_ff['day'].map(days)

# getting the average
feb20_daily_ff = pd.melt(frame = feb20_ff[feb20_ff['area'] == 'CAZ'], id_vars = ['count_date', 'count_week', 'count_month', 'day', 'count_type'],
                         value_vars=['h9','h10','h11','h12','h13','h14','h15','h16','h17','h18'], var_name = 'hour', value_name = 'count_value') \
                    .groupby(by = ['count_month', 'day', 'count_type']).mean(numeric_only = True).reset_index()[['count_month', 'day', 'count_type', 'count_value']]
feb20_daily_ff = feb20_daily_ff.pivot(index = ['count_month', 'day'], columns = 'count_type', values = 'count_value').reset_index()

In [ ]:
may_nov_ff = spark.sql("SELECT day, daynumber, CASE WHEN count_month = '2022-05-01' THEN 'May 2022' \
                                                    WHEN count_month = '2023-11-01' THEN 'Nov 2023' \
                                                    ELSE count_month END count_month, count_week,\
                               hex_id, AVG(resident) AS Resident, AVG(worker) AS Worker, AVG(visitor) AS Visitor\
                        FROM hex_footfall_View\
                        WHERE area = 'CAZ' \
                        AND count_month = '2022-05-01' OR count_month = '2023-11-01'\
                        AND time_indicator in ('09-12', '12-15', '15-18')\
                        GROUP BY hex_id, count_month, count_week, day, daynumber\
                        ORDER BY count_month, daynumber;")

may_nov_ff = may_nov_ff.select('count_month', 'count_week', 'day', 'Resident', 'Visitor', 'Worker').toPandas()
may_nov_ff = may_nov_ff.groupby(by = ['count_month', 'count_week', 'day']).sum(numeric_only = True).reset_index() \
                       .groupby(by = ['count_month', 'day']).mean(numeric_only = True).reset_index()[['count_month', 'day', 'Resident', 'Visitor', 'Worker']]
# week_colours = ['rgb(149, 200, 216)', 'rgb(137, 207, 239)', 'rgb(115, 194, 251)', 'rgb(87, 160, 210)', 'rgb(89, 139, 175)', colours[1], 'rgb(70, 130, 180)']

In [ ]:
may22_daily_ff = may_nov_ff[may_nov_ff['count_month'] == 'May 2022']
nov23_daily_ff = may_nov_ff[may_nov_ff['count_month'] == 'Nov 2023']

def indexer(df: pd.DataFrame):
    for i, cat in enumerate(['Resident', 'Visitor', 'Worker']):
        basis = df[df['day'] == 'Wed'][cat].values[0]
        df[cat] = df.apply(lambda x: x[2+i]/basis * 100, axis = 1)
    day_order = {'Mon': 1, 'Tue': 2, 'Wed': 3, 'Thu': 4, 'Fri': 5, 'Sat': 6, 'Sun': 0}
    df['day_order'] = df['day'].map(day_order)
    df.sort_values(by = 'day_order', ascending = True, inplace = True)
    return df

monthly_daily_ff = pd.concat([indexer(feb20_daily_ff), indexer(may22_daily_ff), indexer(nov23_daily_ff)])
monthly_daily_ff.shape

In [ ]:
for cat in ['Worker', 'Visitor', 'Resident']:
    fig = px.bar(monthly_daily_ff, x = 'count_month', y = cat, color = 'day', barmode = 'group',
                 hover_data =  ['day', 'count_month', cat], labels = {'day': 'Day', 'count_month': 'Count Month', cat: cat}, color_discrete_sequence = week_colours)
    pretty_plot(fig,'<b>Daytime CAZ {} Footfall (mean per day)</b><br>(Indexed; Wed = 100%)'.format(cat), 'Day', 'Average Footfall per Hex')
    size_plot(fig, 600, 1000)
    fig.show()
    # pio.write_image(fig, '<b>Daytime CAZ {} Footfall (mean per day)</b>.png'.format(cat), scale=6, width=1600, height=600)

In [ ]:
monthly_dt_footfall = spark.sql("SELECT count_month, day, daynumber, AVG(resident) AS Resident, AVG(worker) AS Worker, AVG(visitor) AS Visitor\
                                 FROM hex_footfall_View\
                                 WHERE area = 'CAZ' \
                                 AND count_month >= '2023-07-01'\
                                 AND time_indicator in ('06-09', '09-12', '12-15', '15-18')\
                                 GROUP BY count_month, day, daynumber\
                                 ORDER BY count_month, daynumber;")

monthly_dt_footfall = monthly_dt_footfall.select('count_month', 'day', 'Worker', 'Visitor', 'Resident').toPandas()
week_colours = ['rgb(149, 200, 216)', 'rgb(137, 207, 239)', 'rgb(115, 194, 251)', 'rgb(87, 160, 210)', 'rgb(89, 139, 175)', colours[1], 'rgb(70, 130, 180)']

for cat in ['Worker', 'Visitor', 'Resident']:
    fig = px.bar(monthly_dt_footfall, x = 'count_month', y = cat, color = 'day', barmode = 'group',
                 hover_data =  ['day', cat], labels = {'day': 'Day', cat: cat}, color_discrete_sequence = week_colours)
    pretty_plot(fig,'<b>Daytime CAZ {} Footfall (mean per day)</b>'.format(cat), 'Day', 'Average Footfall per Hex')
    size_plot(fig, 600, 1000)
    fig.show()
    pio.write_image(fig, '<b>Daytime CAZ {} Footfall (mean per day)</b>.png'.format(cat), scale=6, width=1600, height=600)

In [ ]:
nov_ts_footfall = spark.sql("SELECT day, daynumber, time_indicator AS timeslot, \
                                    AVG(resident) AS Resident, AVG(worker) AS Worker, AVG(visitor) AS Visitor\
                             FROM hex_footfall_View\
                             WHERE area = 'CAZ' \
                             AND count_month = '2023-11-01'\
                             GROUP BY day, daynumber, time_indicator\
                             ORDER BY daynumber, time_indicator;")

nov_ts_footfall = nov_ts_footfall.toPandas()
fig = px.bar(nov_ts_footfall, x = 'day', y = 'Worker', title = 'Workers in CAZ by Day & Timeslot (Nov 2023)', color= 'timeslot', barmode= 'group')
size_plot(fig, 600, 1500)
fig.show()

In [ ]:
fig = px.bar(nov_ts_footfall, x = 'day', y = 'Visitor', title = 'Visitors in CAZ by Day & Timeslot (Nov 2023)', color= 'timeslot', barmode= 'group')
size_plot(fig, 600, 1500)
fig.show()

#### CAZ Footfall

In [ ]:
hex_tod_footfall = spark.sql("SELECT count_date, count_week, day, CASE WHEN time_indicator IN ('06-09', '09-12', '12-15', '15-18') THEN 'daytime'\
                                                                       ELSE 'nighttime' END AS time_of_day, area,\
                                     AVG(resident) AS Resident, AVG(worker) AS Worker, AVG(visitor) AS Visitor\
                              FROM hex_footfall_View \
                              GROUP BY count_date, count_week, day, time_of_day, area\
                              ORDER BY count_date, time_of_day;")
hex_tod_footfall = hex_tod_footfall.toPandas()
# creating the daily hex footfall dataframes in pandas
hex_mon_todff = hex_tod_footfall[(hex_tod_footfall['day'] == 'Mon') & (hex_tod_footfall['area'] == 'CAZ')]
hex_tue_todff = hex_tod_footfall[(hex_tod_footfall['day'] == 'Tue') & (hex_tod_footfall['area'] == 'CAZ')]
hex_wed_todff = hex_tod_footfall[(hex_tod_footfall['day'] == 'Wed') & (hex_tod_footfall['area'] == 'CAZ')]
hex_thu_todff = hex_tod_footfall[(hex_tod_footfall['day'] == 'Thu') & (hex_tod_footfall['area'] == 'CAZ')]
hex_fri_todff = hex_tod_footfall[(hex_tod_footfall['day'] == 'Fri') & (hex_tod_footfall['area'] == 'CAZ')]

# initializing the plot to show weds & fri footfall at daytime by default
fig = go.Figure()
fig.add_trace(go.Scatter(x = hex_mon_todff[hex_mon_todff['time_of_day'] == 'daytime']['count_date'], y = hex_mon_todff[hex_mon_todff['time_of_day'] == 'daytime']['Worker'], name = 'Mondays', visible = True, line=dict(color = colours[0], width = 0.8)))
fig.add_trace(go.Scatter(x = hex_tue_todff[hex_tue_todff['time_of_day'] == 'daytime']['count_date'], y = hex_tue_todff[hex_tue_todff['time_of_day'] == 'daytime']['Worker'], name = 'Tuesday', visible = True, line=dict(color = colours[1], width = 0.8)))
fig.add_trace(go.Scatter(x = hex_wed_todff[hex_wed_todff['time_of_day'] == 'daytime']['count_date'], y = hex_wed_todff[hex_wed_todff['time_of_day'] == 'daytime']['Worker'], name = 'Wednesdays', visible = True, line=dict(color = colours[7], width = 0.8)))
fig.add_trace(go.Scatter(x = hex_thu_todff[hex_thu_todff['time_of_day'] == 'daytime']['count_date'], y = hex_thu_todff[hex_thu_todff['time_of_day'] == 'daytime']['Worker'], name = 'Thursday', visible = True, line=dict(color = colours[3], width = 0.8)))
fig.add_trace(go.Scatter(x = hex_fri_todff[hex_fri_todff['time_of_day'] == 'daytime']['count_date'], y = hex_fri_todff[hex_fri_todff['time_of_day'] == 'daytime']['Worker'], name = 'Fridays', visible = True, line=dict(color = colours[4], width = 3)))

# plot a dropdown chart showing worker, visitor & res footfall at each three hour period
multi_ddplot(fig, [hex_mon_todff, hex_tue_todff, hex_wed_todff, hex_thu_todff, hex_fri_todff], 'count_date', 'time_of_day', button_list = ['Worker', 'Visitor', 'Resident'], buttonposition = [0.1, 0.4, 0.7], labelposition = [0.07, 0.385, 0.695], buttonheight = 1.15)

# add plot & axis titles
pretty_plot(fig, '<b>CAZ Weekday Footfall by Time of Day (mean per Hex)</b>','Date','Average Footfall')

# show plot
fig.show()

In [ ]:
print('Mondays have a standard deviation of ', hex_mon_todff[hex_mon_todff['time_of_day'] == 'daytime']['Worker'].std())
print('Tuesdays have a standard deviation of ', hex_tue_todff[hex_tue_todff['time_of_day'] == 'daytime']['Worker'].std())
print('Wednesdays have a standard deviation of ', hex_wed_todff[hex_wed_todff['time_of_day'] == 'daytime']['Worker'].std())
print('Thursdays have a standard deviation of ', hex_thu_todff[hex_thu_todff['time_of_day'] == 'daytime']['Worker'].std())
print('Fridays have a standard deviation of ', hex_fri_todff[hex_fri_todff['time_of_day'] == 'daytime']['Worker'].std())

In [ ]:
print('Worker Footfall drops by {} percent at nighttime'.format(np.round(hex_tod_footfall[hex_tod_footfall['area'] == 'CAZ'].pivot(index = 'count_date', columns= ['time_of_day'], values = 'Worker').apply(lambda x: (x[0] - x[1])/x[0] * 100, axis = 1).mean(), 2)))
print('Visitor Footfall drops by {} percent at nighttime'.format(np.round(hex_tod_footfall[hex_tod_footfall['area'] == 'CAZ'].pivot(index = 'count_date', columns= ['time_of_day'], values = 'Visitor').apply(lambda x: (x[0] - x[1])/x[0] * 100, axis = 1).mean(), 2)))
print('Resident Footfall drops by {} percent at nighttime'.format(np.round(hex_tod_footfall[hex_tod_footfall['area'] == 'CAZ'].pivot(index = 'count_date', columns= ['time_of_day'], values = 'Resident').apply(lambda x: (x[0] - x[1])/x[0] * 100, axis = 1).mean(), 2)))

In [ ]:
hex_footfall = spark.sql("SELECT count_date, count_week, day, time_indicator, area, AVG(resident) AS Resident, AVG(worker) AS Worker, AVG(visitor) AS Visitor \
                          FROM hex_footfall_View \
                          GROUP BY count_date, count_week, day, time_indicator, area \
                          ORDER BY count_date, time_indicator;")
hex_footfall = hex_footfall.toPandas()
# creating the daily hex footfall dataframes in pandas
hex_mon_CAZff = hex_footfall[(hex_footfall['day'] == 'Mon') & (hex_footfall['area'] == 'CAZ')]
hex_tue_CAZff = hex_footfall[(hex_footfall['day'] == 'Tue') & (hex_footfall['area'] == 'CAZ')]
hex_wed_CAZff = hex_footfall[(hex_footfall['day'] == 'Wed') & (hex_footfall['area'] == 'CAZ')]
hex_thu_CAZff = hex_footfall[(hex_footfall['day'] == 'Thu') & (hex_footfall['area'] == 'CAZ')]
hex_fri_CAZff = hex_footfall[(hex_footfall['day'] == 'Fri') & (hex_footfall['area'] == 'CAZ')]

fig = go.Figure()
# add traces for each weekday
fig.add_trace(go.Scatter(x = hex_mon_CAZff[hex_mon_CAZff['time_indicator'] == '12-15']['count_date'], y = hex_mon_CAZff[hex_mon_CAZff['time_indicator'] == '12-15']['Worker'], name = 'Mondays', visible = True, line=dict(color = colours[0], width = 0.8)))
fig.add_trace(go.Scatter(x = hex_tue_CAZff[hex_tue_CAZff['time_indicator'] == '12-15']['count_date'], y = hex_tue_CAZff[hex_tue_CAZff['time_indicator'] == '12-15']['Worker'], name = 'Tuesday', visible = True, line=dict(color = colours[1], width = 0.8)))
fig.add_trace(go.Scatter(x = hex_wed_CAZff[hex_wed_CAZff['time_indicator'] == '12-15']['count_date'], y = hex_wed_CAZff[hex_wed_CAZff['time_indicator'] == '12-15']['Worker'], name = 'Wednesdays', visible = True, line=dict(color = colours[3], width = 0.8)))
fig.add_trace(go.Scatter(x = hex_thu_CAZff[hex_thu_CAZff['time_indicator'] == '12-15']['count_date'], y = hex_thu_CAZff[hex_thu_CAZff['time_indicator'] == '12-15']['Worker'], name = 'Thursday', visible = True, line=dict(color = colours[5], width = 0.8)))
fig.add_trace(go.Scatter(x = hex_fri_CAZff[hex_fri_CAZff['time_indicator'] == '12-15']['count_date'], y = hex_fri_CAZff[hex_fri_CAZff['time_indicator'] == '12-15']['Worker'], name = 'Fridays', visible = True, line=dict(color = colours[4], width = 3)))

# plot a dropdown chart showing worker, visitor & res footfall at each three hour period
multi_ddplot(fig, [hex_mon_CAZff, hex_tue_CAZff, hex_wed_CAZff, hex_thu_CAZff, hex_fri_CAZff], 'count_date', 'time_indicator', button_list = ['Worker', 'Visitor', 'Resident'], buttonposition = [0.1, 0.4, 0.7], labelposition = [0.08, 0.385, 0.69], buttonheight = 1.15)

# add plot & axis titles
pretty_plot(fig, '<b>Tri-Hourly CAZ Footfall (mean per Hex)</b>','Date','Average Footfall')

# show plot
fig.show()

In [ ]:
wor_pivot = hex_footfall[hex_footfall['area'] == 'CAZ'].pivot(index = 'count_date', columns = 'time_indicator', values = 'Worker')
vis_pivot = hex_footfall[hex_footfall['area'] == 'CAZ'].pivot(index = 'count_date', columns = 'time_indicator', values = 'Visitor')
res_pivot = hex_footfall[hex_footfall['area'] == 'CAZ'].pivot(index = 'count_date', columns = 'time_indicator', values = 'Resident')

# check time_indicator pivoted columns sequence before running the following computation
print('Worker Footfall changes by {}% after 3AM, {}% after 6AM, {}% after 9AM, {}% after noon, {}% after 3PM, {}% after 6PM, {}% after 9PM & {}% after midnight'.format(np.round(wor_pivot.apply(lambda x: (1 - (x[0]/x[1])) * 100, axis = 1).mean(), 2),
                                                                                                                                                                                      np.round(wor_pivot.apply(lambda x: (1 - (x[1]/x[2])) * 100, axis = 1).mean(), 2),
                                                                                                                                                                                      np.round(wor_pivot.apply(lambda x: (1 - (x[2]/x[3])) * 100, axis = 1).mean(), 2),
                                                                                                                                                                                      np.round(wor_pivot.apply(lambda x: (1 - (x[3]/x[4])) * 100, axis = 1).mean(), 2),
                                                                                                                                                                                      np.round(wor_pivot.apply(lambda x: (1 - (x[4]/x[5])) * 100, axis = 1).mean(), 2),
                                                                                                                                                                                      np.round(wor_pivot.apply(lambda x: (1 - (x[5]/x[6])) * 100, axis = 1).mean(), 2),
                                                                                                                                                                                      np.round(wor_pivot.apply(lambda x: (1 - (x[6]/x[7])) * 100, axis = 1).mean(), 2),
                                                                                                                                                                                      np.round(wor_pivot.apply(lambda x: (1 - (x[7]/x[0])) * 100, axis = 1).mean(), 2)))
print('Visitor Footfall changes by {}% after 3AM, {}% after 6AM, {}% after 9AM, {}% after noon, {}% after 3PM, {}% after 6PM, {}% after 9PM & {}% after midnight'.format(np.round(vis_pivot.apply(lambda x: (1 - (x[0]/x[1])) * 100, axis = 1).mean(), 2),
                                                                                                                                                                                        np.round(vis_pivot.apply(lambda x: (1 - (x[1]/x[2])) * 100, axis = 1).mean(), 2),
                                                                                                                                                                                        np.round(vis_pivot.apply(lambda x: (1 - (x[2]/x[3])) * 100, axis = 1).mean(), 2),
                                                                                                                                                                                        np.round(vis_pivot.apply(lambda x: (1 - (x[3]/x[4])) * 100, axis = 1).mean(), 2),
                                                                                                                                                                                        np.round(vis_pivot.apply(lambda x: (1 - (x[4]/x[5])) * 100, axis = 1).mean(), 2),
                                                                                                                                                                                        np.round(vis_pivot.apply(lambda x: (1 - (x[5]/x[6])) * 100, axis = 1).mean(), 2),
                                                                                                                                                                                        np.round(vis_pivot.apply(lambda x: (1 - (x[6]/x[7])) * 100, axis = 1).mean(), 2),
                                                                                                                                                                                        np.round(vis_pivot.apply(lambda x: (1 - (x[7]/x[0])) * 100, axis = 1).mean(), 2)))
print('Resident Footfall changes by {}% after 3AM, {}% after 6AM, {}% after 9AM, {}% after noon, {}% after 3PM, {}% after 6PM, {}% after 9PM & {}% after midnight'.format(np.round(res_pivot.apply(lambda x: (1 - (x[0]/x[1])) * 100, axis = 1).mean(), 2),
                                                                                                                                                                                          np.round(res_pivot.apply(lambda x: (1 - (x[1]/x[2])) * 100, axis = 1).mean(), 2),
                                                                                                                                                                                          np.round(res_pivot.apply(lambda x: (1 - (x[2]/x[3])) * 100, axis = 1).mean(), 2),
                                                                                                                                                                                          np.round(res_pivot.apply(lambda x: (1 - (x[3]/x[4])) * 100, axis = 1).mean(), 2),
                                                                                                                                                                                          np.round(res_pivot.apply(lambda x: (1 - (x[4]/x[5])) * 100, axis = 1).mean(), 2),
                                                                                                                                                                                          np.round(res_pivot.apply(lambda x: (1 - (x[5]/x[6])) * 100, axis = 1).mean(), 2),
                                                                                                                                                                                          np.round(res_pivot.apply(lambda x: (1 - (x[6]/x[7])) * 100, axis = 1).mean(), 2),
                                                                                                                                                                                          np.round(res_pivot.apply(lambda x: (1 - (x[7]/x[0])) * 100, axis = 1).mean(), 2)))

#### CAZ vs Non-CAZ Footfall

In [ ]:
monthly_dt_NCAZ_footfall = spark.sql("SELECT count_month, day, daynumber, AVG(resident) AS Resident, AVG(worker) AS Worker, AVG(visitor) AS Visitor\
                                      FROM hex_footfall_View\
                                      WHERE area = 'Non-CAZ' \
                                      AND count_month >= '2023-07-01'\
                                      AND time_indicator in ('06-09', '09-12', '12-15', '15-18')\
                                      GROUP BY count_month, day, daynumber\
                                      ORDER BY count_month, daynumber;")

monthly_dt_NCAZ_footfall = monthly_dt_NCAZ_footfall.select('count_month', 'day', 'Worker', 'Visitor', 'Resident').toPandas()

for cat in ['Worker', 'Visitor', 'Resident']:
    fig = px.bar(monthly_dt_NCAZ_footfall, x = 'count_month', y = cat, color = 'day', barmode = 'group',
                 hover_data =  ['day', cat], labels = {'day': 'Day', cat: cat}, color_discrete_sequence = week_colours)
    pretty_plot(fig,'<b>Daytime Non-CAZ {} Footfall (mean per day)</b>'.format(cat), 'Day', 'Average Footfall per Hex')
    size_plot(fig, 600, 1000)
    fig.show()

In [ ]:
# To compare daytime CAZ and Non-CAZ worker footfall, we need to pivot the day column and aggregate the first value in the Worker column
hex_CAZ_wor_todff = hex_tod_footfall[hex_tod_footfall['area'] == 'CAZ'].pivot(index = ['count_week', 'time_of_day'], columns = 'day', values = 'Worker').reset_index().sort_values(by = 'count_week')
hex_NCAZ_wor_todff = hex_tod_footfall[hex_tod_footfall['area'] == 'Non-CAZ'].pivot(index = ['count_week', 'time_of_day'], columns = 'day', values = 'Worker').reset_index().sort_values(by = 'count_week')

# initializing the plot
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x = hex_CAZ_wor_todff[hex_CAZ_wor_todff['time_of_day'] == 'daytime']['count_week'], y = hex_CAZ_wor_todff[hex_CAZ_wor_todff['time_of_day'] == 'daytime']['Mon'], name = 'CAZ', visible = True), secondary_y = False)
fig.add_trace(go.Scatter(x = hex_NCAZ_wor_todff[hex_NCAZ_wor_todff['time_of_day'] == 'daytime']['count_week'], y = hex_NCAZ_wor_todff[hex_NCAZ_wor_todff['time_of_day'] == 'daytime']['Mon'], name = 'Non-CAZ', visible = True), secondary_y = True)

# plot a dropdown chart showing worker, visitor & res footfall at each three hour period
multi_ddplot(fig, [hex_CAZ_wor_todff, hex_NCAZ_wor_todff], 'count_week', 'time_of_day', button_list = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'], buttonposition = [0.015, 0.165, 0.315, 0.465, 0.615, 0.765, 0.915], labelposition = [0, 0.152, 0.3, 0.457, 0.61, 0.76, 0.91], buttonheight = 1.15)

# add plot & axis titles
pretty_plot(fig, '<b>Daytime & Nighttime Worker Footfall in CAZ & Non-CAZ (mean per Hex)</b>','Date','Average Footfall')

# show plot
fig.show()

In [ ]:
# To compare daytime CAZ and Non-CAZ worker footfall, we need to pivot the day column and aggregate the first value in the Worker column
hex_CAZ_worff = hex_footfall[hex_footfall['area'] == 'CAZ'].pivot(index = ['count_week', 'time_indicator'], columns = 'day', values = 'Worker').reset_index().sort_values(by = ['count_week', 'time_indicator'])
hex_NCAZ_worff = hex_footfall[hex_footfall['area'] == 'Non-CAZ'].pivot(index = ['count_week', 'time_indicator'], columns = 'day', values = 'Worker').reset_index().sort_values(by = ['count_week', 'time_indicator'])

# initializing the plot
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x = hex_CAZ_worff[hex_CAZ_worff['time_indicator'] == '00-03']['count_week'], y = hex_CAZ_worff[hex_CAZ_worff['time_indicator'] == '00-03']['Mon'], name = 'CAZ', visible = True))
fig.add_trace(go.Scatter(x = hex_NCAZ_worff[hex_NCAZ_worff['time_indicator'] == '00-03']['count_week'], y = hex_NCAZ_worff[hex_NCAZ_worff['time_indicator'] == '00-03']['Mon'], name = 'Non-CAZ', visible = True), secondary_y = True)

# plot a dropdown chart comparingCAZ & Non-CAZ worker footfall at each three hour period on each day of the week
multi_ddplot(fig, [hex_CAZ_worff, hex_NCAZ_worff], 'count_week', 'time_indicator', button_list = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'], buttonposition = [0.015, 0.165, 0.315, 0.465, 0.615, 0.765, 0.915], labelposition = [0, 0.152, 0.3, 0.457, 0.61, 0.76, 0.91], buttonheight = 1.15)

# add plot & axis titles
pretty_plot(fig, '<b>Tri-Hourly Worker Footfall in CAZ & Non-CAZ (mean per Hex)</b>','Date','Average Footfall')

# show plot
fig.show()

In [ ]:
hex_CAZ_worff2 = hex_tod_footfall[(hex_tod_footfall['area'] == 'CAZ') & (hex_tod_footfall['time_of_day'] == 'daytime')][['count_date', 'Worker']]
hex_CAZ_resff = hex_tod_footfall[(hex_tod_footfall['area'] == 'CAZ') & (hex_tod_footfall['time_of_day'] == 'daytime')][['count_date', 'Resident']]
hex_NCAZ_worff = hex_tod_footfall[(hex_tod_footfall['area'] == 'Non-CAZ') & (hex_tod_footfall['time_of_day'] == 'daytime')][['count_date', 'Worker']]
hex_NCAZ_resff2 = hex_tod_footfall[(hex_tod_footfall['area'] == 'Non-CAZ') & (hex_tod_footfall['time_of_day'] == 'daytime')][['count_date', 'Resident']]

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x = hex_CAZ_worff2['count_date'], y = hex_CAZ_worff2['Worker'], name = 'CAZ Worker', visible = True))
fig.add_trace(go.Scatter(x = hex_NCAZ_resff2['count_date'], y = hex_NCAZ_resff2['Resident'], name = 'Non-CAZ Resident', visible = True), secondary_y= True)

pretty_plot(fig, '<b>Daytime Worker Footfall in CAZ vs Resident Footfall in Non-CAZ (mean per Hex)</b>','Date','Average Footfall')
fig.show()

In [ ]:
hex_CAZ_visff = hex_tod_footfall[(hex_tod_footfall['area'] == 'CAZ') & (hex_tod_footfall['time_of_day'] == 'daytime')][['count_date', 'Visitor']]
corr = pd.DataFrame(pd.date_range(start = "2022-05-01", end = "2023-12-27"), columns=['count_date'])
hex_CAZ_visff['count_date'] = pd.to_datetime(hex_CAZ_visff['count_date'])
hex_CAZ_worff2['count_date'] = pd.to_datetime(hex_CAZ_worff2['count_date'])
hex_CAZ_resff['count_date'] = pd.to_datetime(hex_CAZ_resff['count_date'])
hex_NCAZ_worff['count_date'] = pd.to_datetime(hex_NCAZ_worff['count_date'])
hex_NCAZ_resff2['count_date'] = pd.to_datetime(hex_NCAZ_resff2['count_date'])
corr = corr.merge(right = hex_CAZ_visff, on = 'count_date', how = 'left')\
           .merge(right = hex_CAZ_worff2, on = 'count_date', how = 'left')\
           .merge(right = hex_CAZ_resff, on = 'count_date', how = 'left')\
           .merge(right = hex_NCAZ_worff, on = 'count_date', how = 'left')\
           .merge(right = hex_NCAZ_resff2, on = 'count_date', how = 'left')
corr.columns = ['count_date', 'CAZ Visitor', 'CAZ Worker', 'CAZ Resident', 'Non-CAZ Worker', 'Non-CAZ Resident']

import seaborn as sns
matrix = np.triu(corr[['CAZ Worker', 'CAZ Resident', 'Non-CAZ Worker', 'Non-CAZ Resident']].corr())
# using the upper triangle matrix as mask
sns.heatmap(corr[['CAZ Worker', 'CAZ Resident', 'Non-CAZ Worker', 'Non-CAZ Resident']].corr(), annot=True, mask=matrix)

In [ ]:
hex_NCAZ_res_todff = hex_tod_footfall[hex_tod_footfall['area'] == 'Non-CAZ'].pivot(index = ['count_week', 'time_of_day'], columns = 'day', values = 'Resident').reset_index().sort_values(by = 'count_week')

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x = hex_CAZ_wor_todff[hex_CAZ_wor_todff['time_of_day'] == 'daytime']['count_week'], y = hex_CAZ_wor_todff[hex_CAZ_wor_todff['time_of_day'] == 'daytime']['Mon'], name = 'CAZ Worker', visible = True))
fig.add_trace(go.Scatter(x = hex_NCAZ_res_todff[hex_NCAZ_res_todff['time_of_day'] == 'daytime']['count_week'], y = hex_NCAZ_res_todff[hex_NCAZ_res_todff['time_of_day'] == 'daytime']['Mon'], name = 'Non-CAZ Resident', visible = True), secondary_y= True)

# plot a dropdown chart showing worker, visitor & res footfall at each three hour period
multi_ddplot(fig, [hex_CAZ_wor_todff, hex_NCAZ_res_todff], 'count_week', 'time_of_day', button_list = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'], buttonposition = [0.015, 0.165, 0.315, 0.465, 0.615, 0.765, 0.915], labelposition = [0, 0.152, 0.3, 0.457, 0.61, 0.76, 0.91], buttonheight = 1.15)

# add plot & axis titles
pretty_plot(fig, '<b>Worker Footfall in CAZ vs Resident Footfall in Non-CAZ (mean per Hex)</b>','Date','Average Footfall')

# show plot
fig.show()

In [ ]:
hex_NCAZ_resff = hex_footfall[hex_footfall['area'] == 'Non-CAZ']\
                             .pivot(index = ['count_week', 'time_indicator'], columns = 'day', values = 'Resident')\
                             .reset_index()\
                             .sort_values(by= ['count_week', 'time_indicator'])

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x = hex_CAZ_worff[hex_CAZ_worff['time_indicator'] == '00-03']['count_week'], y = hex_CAZ_worff[hex_CAZ_worff['time_indicator'] == '00-03']['Mon'], name = 'CAZ Worker', visible = True))
fig.add_trace(go.Scatter(x = hex_NCAZ_resff[hex_NCAZ_resff['time_indicator'] == '00-03']['count_week'], y = hex_NCAZ_resff[hex_NCAZ_resff['time_indicator'] == '00-03']['Mon'], name = 'Non-CAZ Resident', visible = True), secondary_y= True)

# plot a dropdown chart showing worker, visitor & res footfall at each three hour period
multi_ddplot(fig, [hex_CAZ_worff, hex_NCAZ_resff], 'count_week', 'time_indicator', button_list = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'], buttonposition = [0.015, 0.165, 0.315, 0.465, 0.615, 0.765, 0.915], labelposition = [0, 0.152, 0.3, 0.457, 0.61, 0.76, 0.91], buttonheight = 1.15)

# add plot & axis titles
pretty_plot(fig, '<b>Tri-Hourly Worker Footfall in CAZ vs Resident Footfall in Non-CAZ (mean per Hex)</b>','Date','Average Footfall')

# show plot
fig.show()

### **Spend Analysis**

In [ ]:
spend.createOrReplaceTempView('spend_view')
spend.show(5)

#### CAZ Spend

In [ ]:
fy_23_spend = spark.sql("SELECT txn_month, area, SUM(txn_amt) AS Spend_Value, SUM(txn_cnt) AS Spend_Frequency\
                            FROM spend_view\
                            WHERE txn_month >= '2023-01-01'\
                            GROUP BY txn_month, area\
                            ORDER BY txn_month, area;")

fy_23_spend = fy_23_spend.toPandas()

fig = px.bar(fy_23_spend, x = 'txn_month', y = 'Spend_Value', color = 'area', barmode = 'group', color_discrete_sequence = colours[:2],
             hover_data=['txn_month', 'area', 'Spend_Value'], labels={'txn_month': 'Month', 'area': 'Area', 'Spend_Value': 'Spend Value'})
pretty_plot(fig,'<b>Monthly Total Transaction Value in 2023</b>', 'Month', 'Transactions')
size_plot(fig, 600, 1200)
fig.show()

In [ ]:
fig = px.bar(fy_23_spend, x = 'txn_month', y = 'Spend_Frequency', color = 'area', barmode = 'group', color_discrete_sequence = colours[3:5],
             hover_data = ['txn_month', 'area', 'Spend_Frequency'], labels = {'txn_month': 'Month', 'area': 'Area', 'Spend_Frequency': 'Spend Frequency'})
pretty_plot(fig,'<b>Monthly Transaction Frequency in 2023</b>', 'Month', 'Transactions')
size_plot(fig, 600, 1200)
fig.show()

In [ ]:
monthly_dt_spend = spark.sql("SELECT txn_month, day, daynumber, year, time_of_day, AVG(Spend_Value) AS Spend_Value, AVG(Spend_Frequency) AS Spend_Frequency\
                              FROM (SELECT txn_month, txn_week, day, daynumber, YEAR(count_date) AS year, SUM(txn_amt) AS Spend_Value, SUM(txn_cnt) AS Spend_Frequency,\
                                    CASE WHEN hours in ('06-09', '09-12', '12-15', '15-18') THEN 'daytime'\
                                        ELSE 'nighttime' END AS time_of_day\
                                    FROM spend_view\
                                    WHERE area = 'CAZ' \
                                    GROUP BY txn_month, txn_week, day, daynumber, time_of_day, year)\
                              GROUP BY txn_month, day, daynumber, time_of_day, year\
                              ORDER BY txn_month, daynumber;")

monthly_dt_spend = monthly_dt_spend.select('txn_month', 'day', 'time_of_day', 'year', 'Spend_Value', 'Spend_Frequency').toPandas()

for i, year in enumerate(monthly_dt_spend.year.unique()):
    fig = px.bar(monthly_dt_spend[(monthly_dt_spend['year'] == year) & (monthly_dt_spend['time_of_day'] == 'daytime')], x = 'txn_month', y = 'Spend_Value', color = 'day', barmode = 'group',
                 hover_data =  ['day', 'Spend_Value', 'Spend_Frequency'], labels = {'day': 'Day', 'Spend_Value': 'Spend Value', 'Spend_Frequency': 'Spend Frequency'}, color_discrete_sequence = week_colours)
    pretty_plot(fig,'<b>Daytime CAZ Spend in {}</b>'.format(year), 'Month', 'Transactions')
    size_plot(fig, 600, 1500)
    fig.show()

In [ ]:
week_colours_f = ['rgb(236, 245, 233)', 'rgb(219, 235, 213)', 'rgb(202, 225, 194)', 'rgb(186, 215, 176)', 'rgb(171, 205, 158)', colours[4], 'rgb(156, 195, 142)']
for i, year in enumerate(monthly_dt_spend.year.unique()):
    fig = px.bar(monthly_dt_spend[(monthly_dt_spend['year'] == year) & (monthly_dt_spend['time_of_day'] == 'daytime')], x = 'txn_month', y = 'Spend_Frequency', color = 'day', barmode = 'group',
                 hover_data =  ['day', 'Spend_Frequency'], labels = {'day': 'Day', 'Spend_Frequency': 'Spend Frequency'}, color_discrete_sequence = week_colours_f)
    pretty_plot(fig,'<b>Daytime CAZ Spend Frequency in {}</b>'.format(year), 'Month', 'Transactions')
    size_plot(fig, 600, 1500)
    fig.show()

In [ ]:
for i, year in enumerate(monthly_dt_spend.year.unique()):
    fig = px.bar(monthly_dt_spend[(monthly_dt_spend['year'] == year) & (monthly_dt_spend['time_of_day'] == 'nighttime')], x = 'txn_month', y = 'Spend_Value', color = 'day', barmode = 'group',
                 hover_data =  ['day', 'Spend_Value', 'Spend_Frequency'], labels = {'day': 'Day', 'Spend_Value': 'Spend Value', 'Spend_Frequency': 'Spend Frequency'}, color_discrete_sequence = week_colours)
    pretty_plot(fig,'<b>Nighttime CAZ Spend in {}</b>'.format(year), 'Month', 'Transactions')
    size_plot(fig, 600, 1500)
    fig.show()

In [ ]:
for i, year in enumerate(monthly_dt_spend.year.unique()):
    fig = px.bar(monthly_dt_spend[(monthly_dt_spend['year'] == year) & (monthly_dt_spend['time_of_day'] == 'nighttime')], x = 'txn_month', y = 'Spend_Frequency', color = 'day', barmode = 'group',
                 hover_data =  ['day', 'Spend_Frequency'], labels = {'day': 'Day', 'Spend_Frequency': 'Spend Frequency'}, color_discrete_sequence = week_colours_f)
    pretty_plot(fig,'<b>Nighttime CAZ Spend Frequency in {}</b>'.format(year), 'Month', 'Transactions')
    size_plot(fig, 600, 1500)
    fig.show()

In [ ]:
hex_spend = spark.sql("SELECT count_date, txn_week, day, \
                              CASE WHEN hours in ('06-09', '09-12', '12-15', '15-18') THEN 'daytime' ELSE 'nighttime' END time_of_day, \
                              SUM(txn_amt) AS Spend_Value, SUM(txn_cnt) AS Spend_Frequency \
                        FROM spend_view\
                        WHERE area = 'CAZ' \
                        GROUP BY count_date, txn_week, day, time_of_day \
                        ORDER BY count_date, txn_week, time_of_day;")
hex_spend = hex_spend.toPandas()

hex_mon_spend = hex_spend[hex_spend['day'] == 'Mon'][['count_date', 'time_of_day', 'Spend_Value', 'Spend_Frequency']]
hex_tue_spend = hex_spend[hex_spend['day'] == 'Tue'][['count_date', 'time_of_day', 'Spend_Value', 'Spend_Frequency']]
hex_wed_spend = hex_spend[hex_spend['day'] == 'Wed'][['count_date', 'time_of_day', 'Spend_Value', 'Spend_Frequency']]
hex_thu_spend = hex_spend[hex_spend['day'] == 'Thu'][['count_date', 'time_of_day', 'Spend_Value', 'Spend_Frequency']]
hex_fri_spend = hex_spend[hex_spend['day'] == 'Fri'][['count_date', 'time_of_day', 'Spend_Value', 'Spend_Frequency']]
hex_sat_spend = hex_spend[hex_spend['day'] == 'Sat'][['count_date', 'time_of_day', 'Spend_Value', 'Spend_Frequency']]
hex_sun_spend = hex_spend[hex_spend['day'] == 'Sun'][['count_date', 'time_of_day', 'Spend_Value', 'Spend_Frequency']]

fig = go.Figure()
fig.add_trace(go.Scatter(x = hex_mon_spend[hex_mon_spend['time_of_day'] == 'daytime']['count_date'], y = hex_mon_spend[hex_mon_spend['time_of_day'] == 'daytime']['Spend_Value'], name = 'Monday', visible = True, line=dict(color = colours[0], width = 0.8)))
fig.add_trace(go.Scatter(x = hex_tue_spend[hex_tue_spend['time_of_day'] == 'daytime']['count_date'], y = hex_tue_spend[hex_tue_spend['time_of_day'] == 'daytime']['Spend_Value'], name = 'Tuesday', visible = True, line=dict(color = colours[1], width = 0.8)))
fig.add_trace(go.Scatter(x = hex_wed_spend[hex_wed_spend['time_of_day'] == 'daytime']['count_date'], y = hex_wed_spend[hex_wed_spend['time_of_day'] == 'daytime']['Spend_Value'], name = 'Wednesday', visible = True, line=dict(color = colours[7], width = 0.8)))
fig.add_trace(go.Scatter(x = hex_thu_spend[hex_thu_spend['time_of_day'] == 'daytime']['count_date'], y = hex_thu_spend[hex_thu_spend['time_of_day'] == 'daytime']['Spend_Value'], name = 'Thursday', visible = True, line=dict(color = colours[3], width = 0.8)))
fig.add_trace(go.Scatter(x = hex_fri_spend[hex_fri_spend['time_of_day'] == 'daytime']['count_date'], y = hex_fri_spend[hex_fri_spend['time_of_day'] == 'daytime']['Spend_Value'], name = 'Friday', visible = True, line=dict(color = colours[4], width = 3)))
fig.add_trace(go.Scatter(x = hex_sat_spend[hex_sat_spend['time_of_day'] == 'daytime']['count_date'], y = hex_sat_spend[hex_sat_spend['time_of_day'] == 'daytime']['Spend_Value'], name = 'Saturday', visible = True, line=dict(color = colours[5], width = 0.8)))
fig.add_trace(go.Scatter(x = hex_sun_spend[hex_sun_spend['time_of_day'] == 'daytime']['count_date'], y = hex_sun_spend[hex_sun_spend['time_of_day'] == 'daytime']['Spend_Value'], name = 'Sunday', visible = True, line=dict(color = colours[6], width = 0.8)))

multi_ddplot(fig, [hex_mon_spend, hex_tue_spend, hex_wed_spend, hex_thu_spend, hex_fri_spend, hex_sat_spend, hex_sun_spend], 'count_date', 'time_of_day', button_list = ['Spend_Value', 'Spend_Frequency'], buttonposition = [0.1, 0.7], labelposition = [0.04, 0.69], buttonheight = 1.15)

pretty_plot(fig, '<b>CAZ Transactions (Sum)</b>','Date','Transactions')
fig.show()

In [ ]:
hex_dt_spend = pd.melt(hex_spend[hex_spend['time_of_day'] == 'daytime'], id_vars=['txn_week', 'day'],
                       value_vars=['Spend_Value', 'Spend_Frequency'], var_name = 'cnt_type', value_name = 'cnt_value')\
                 .pivot(index = ['txn_week', 'cnt_type'], columns='day', values='cnt_value').reset_index()
hex_nt_spend = pd.melt(hex_spend[hex_spend['time_of_day'] == 'nighttime'], id_vars=['txn_week', 'day'],
                       value_vars=['Spend_Value','Spend_Frequency'], var_name = 'cnt_type', value_name = 'cnt_value')\
                 .pivot(index = ['txn_week', 'cnt_type'], columns='day', values='cnt_value').reset_index()
hex_spend_comp = hex_dt_spend.merge(hex_nt_spend, how = 'left', on = ['txn_week', 'cnt_type'], suffixes = ('_dt', '_nt'))
hex_spend_comp['Mon_var_pc'] = np.round(hex_spend_comp['Mon_nt']/hex_spend_comp['Mon_dt'] * 100, 2)
hex_spend_comp['Tue_var_pc'] = np.round(hex_spend_comp['Tue_nt']/hex_spend_comp['Tue_dt'] * 100, 2)
hex_spend_comp['Wed_var_pc'] = np.round(hex_spend_comp['Wed_nt']/hex_spend_comp['Wed_dt'] * 100, 2)
hex_spend_comp['Thu_var_pc'] = np.round(hex_spend_comp['Thu_nt']/hex_spend_comp['Thu_dt'] * 100, 2)
hex_spend_comp['Fri_var_pc'] = np.round(hex_spend_comp['Fri_nt']/hex_spend_comp['Fri_dt'] * 100, 2)

fig = go.Figure()
fig.add_trace(go.Scatter(x = hex_spend_comp[hex_spend_comp['cnt_type'] == 'Spend_Value']['txn_week'], y = hex_spend_comp[hex_spend_comp['cnt_type'] == 'Spend_Value']['Mon_var_pc'], name = 'Monday', visible = True, line=dict(color = colours[0], width = 0.8)))
fig.add_trace(go.Scatter(x = hex_spend_comp[hex_spend_comp['cnt_type'] == 'Spend_Value']['txn_week'], y = hex_spend_comp[hex_spend_comp['cnt_type'] == 'Spend_Value']['Tue_var_pc'], name = 'Tuesday', visible = True, line=dict(color = colours[1], width = 0.8)))
fig.add_trace(go.Scatter(x = hex_spend_comp[hex_spend_comp['cnt_type'] == 'Spend_Value']['txn_week'], y = hex_spend_comp[hex_spend_comp['cnt_type'] == 'Spend_Value']['Wed_var_pc'], name = 'Wednesday', visible = True, line=dict(color = colours[7], width = 0.8)))
fig.add_trace(go.Scatter(x = hex_spend_comp[hex_spend_comp['cnt_type'] == 'Spend_Value']['txn_week'], y = hex_spend_comp[hex_spend_comp['cnt_type'] == 'Spend_Value']['Thu_var_pc'], name = 'Thursday', visible = True, line=dict(color = colours[3], width = 0.8)))
fig.add_trace(go.Scatter(x = hex_spend_comp[hex_spend_comp['cnt_type'] == 'Spend_Value']['txn_week'], y = hex_spend_comp[hex_spend_comp['cnt_type'] == 'Spend_Value']['Fri_var_pc'], name = 'Friday', visible = True, line=dict(color = colours[4], width = 3)))
buttons = []
# button with one option for each dataframe
for col in ['Spend_Value','Spend_Frequency']:
    x = []
    y = []
    traces = []
    for j, day in enumerate(['Mon', 'Tue', 'Wed', 'Thu', 'Fri']):
        x.append(hex_spend_comp[hex_spend_comp['cnt_type'] == col]['txn_week'])
        y.append(hex_spend_comp[hex_spend_comp['cnt_type'] == col][day+'_var_pc'])
        traces.append(j)
    buttons.append(dict(method='restyle',
                        label=col,
                        visible=True,
                        args=[{'y': y,
                               'x': x,
                               'type': 'scatter'}, traces]))

# some adjustments to the updatemenus
updatemenu = [dict(buttons = buttons, direction = 'down', showactive = True, type = 'buttons', yanchor = 'top')]
fig.update_layout(showlegend = True, updatemenus = updatemenu)
pretty_plot(fig, '<b>Nighttime Spend Indexed to Daytime Spend</b>', 'Week', 'Nighttime Indexed to Daytime')
fig.show()

In [ ]:
hex_tsspend = spark.sql("SELECT count_date, txn_week, day, daynumber, hours, \
                                SUM(txn_amt) AS Spend_Value, SUM(txn_cnt) AS Spend_Frequency \
                        FROM Spend_View\
                        WHERE area = 'CAZ' \
                        GROUP BY count_date, txn_week, day, daynumber, hours \
                        ORDER BY count_date, txn_week, daynumber, hours;")
hex_tsspend = hex_tsspend.toPandas()

hex_mon_tsspend = hex_tsspend[hex_tsspend['day'] == 'Mon'][['count_date', 'hours', 'Spend_Value', 'Spend_Frequency']]
hex_tue_tsspend = hex_tsspend[hex_tsspend['day'] == 'Tue'][['count_date', 'hours', 'Spend_Value', 'Spend_Frequency']]
hex_wed_tsspend = hex_tsspend[hex_tsspend['day'] == 'Wed'][['count_date', 'hours', 'Spend_Value', 'Spend_Frequency']]
hex_thu_tsspend = hex_tsspend[hex_tsspend['day'] == 'Thu'][['count_date', 'hours', 'Spend_Value', 'Spend_Frequency']]
hex_fri_tsspend = hex_tsspend[hex_tsspend['day'] == 'Fri'][['count_date', 'hours', 'Spend_Value', 'Spend_Frequency']]

fig = go.Figure()
fig.add_trace(go.Scatter(x = hex_mon_tsspend[hex_mon_tsspend['hours'] == '00-03']['count_date'], y = hex_mon_tsspend[hex_mon_tsspend['hours'] == '00-03']['Spend_Value'], name = 'Monday', visible = True))
fig.add_trace(go.Scatter(x = hex_tue_tsspend[hex_tue_tsspend['hours'] == '00-03']['count_date'], y = hex_tue_tsspend[hex_tue_tsspend['hours'] == '00-03']['Spend_Value'], name = 'Tuesday', visible = True))
fig.add_trace(go.Scatter(x = hex_wed_tsspend[hex_wed_tsspend['hours'] == '00-03']['count_date'], y = hex_wed_tsspend[hex_wed_tsspend['hours'] == '00-03']['Spend_Value'], name = 'Wednesday', visible = True))
fig.add_trace(go.Scatter(x = hex_thu_tsspend[hex_thu_tsspend['hours'] == '00-03']['count_date'], y = hex_thu_tsspend[hex_thu_tsspend['hours'] == '00-03']['Spend_Value'], name = 'Thursday', visible = True))
fig.add_trace(go.Scatter(x = hex_fri_tsspend[hex_fri_tsspend['hours'] == '00-03']['count_date'], y = hex_fri_tsspend[hex_fri_tsspend['hours'] == '00-03']['Spend_Value'], name = 'Friday', visible = True, line=dict(color = colours[4], width = 3)))

multi_ddplot(fig, [hex_mon_tsspend, hex_tue_tsspend, hex_wed_tsspend, hex_thu_tsspend, hex_fri_tsspend], 'count_date', 'hours',
             button_list = ['Spend_Value', 'Spend_Frequency'], buttonposition = [0.1, 0.6], labelposition = [0.03, 0.55], buttonheight = 1.15)

pretty_plot(fig, '<b>Tri-Hourly CAZ Transactions per Day (Sum)</b>','Date','Transactions')
fig.show()

In [ ]:
hex_0003_spend = hex_tsspend[hex_tsspend['hours'] == '00-03'][['txn_week', 'day', 'Spend_Value', 'Spend_Frequency']]
hex_0306_spend = hex_tsspend[hex_tsspend['hours'] == '03-06'][['txn_week', 'day', 'Spend_Value', 'Spend_Frequency']]
hex_0609_spend = hex_tsspend[hex_tsspend['hours'] == '06-09'][['txn_week', 'day', 'Spend_Value', 'Spend_Frequency']]
hex_0912_spend = hex_tsspend[hex_tsspend['hours'] == '09-12'][['txn_week', 'day', 'Spend_Value', 'Spend_Frequency']]
hex_1215_spend = hex_tsspend[hex_tsspend['hours'] == '12-15'][['txn_week', 'day', 'Spend_Value', 'Spend_Frequency']]
hex_1518_spend = hex_tsspend[hex_tsspend['hours'] == '15-18'][['txn_week', 'day', 'Spend_Value', 'Spend_Frequency']]
hex_1821_spend = hex_tsspend[hex_tsspend['hours'] == '18-21'][['txn_week', 'day', 'Spend_Value', 'Spend_Frequency']]
hex_2124_spend = hex_tsspend[hex_tsspend['hours'] == '21-24'][['txn_week', 'day', 'Spend_Value', 'Spend_Frequency']]

fig = go.Figure()
fig.add_trace(go.Scatter(x = hex_0003_spend[hex_0003_spend['day'] == 'Mon']['txn_week'], y = hex_0003_spend[hex_0003_spend['day'] == 'Mon']['Spend_Value'],
                         name = '00-03', visible = True))
fig.add_trace(go.Scatter(x = hex_0306_spend[hex_0306_spend['day'] == 'Mon']['txn_week'], y = hex_0306_spend[hex_0306_spend['day'] == 'Mon']['Spend_Value'],
                         name = '03-06', visible = True))
fig.add_trace(go.Scatter(x = hex_0609_spend[hex_0609_spend['day'] == 'Mon']['txn_week'], y = hex_0609_spend[hex_0609_spend['day'] == 'Mon']['Spend_Value'],
                         name = '06-09', visible = True))
fig.add_trace(go.Scatter(x = hex_0912_spend[hex_0912_spend['day'] == 'Mon']['txn_week'], y = hex_0912_spend[hex_0912_spend['day'] == 'Mon']['Spend_Value'],
                         name = '09-12', visible = True))
fig.add_trace(go.Scatter(x = hex_1215_spend[hex_1215_spend['day'] == 'Mon']['txn_week'], y = hex_1215_spend[hex_1215_spend['day'] == 'Mon']['Spend_Value'],
                         name = '12-15', visible = True))
fig.add_trace(go.Scatter(x = hex_1518_spend[hex_1518_spend['day'] == 'Mon']['txn_week'], y = hex_1518_spend[hex_1518_spend['day'] == 'Mon']['Spend_Value'],
                         name = '15-18', visible = True))
fig.add_trace(go.Scatter(x = hex_1821_spend[hex_1821_spend['day'] == 'Mon']['txn_week'], y = hex_1821_spend[hex_1821_spend['day'] == 'Mon']['Spend_Value'],
                         name = '18-21', visible = True))
fig.add_trace(go.Scatter(x = hex_2124_spend[hex_2124_spend['day'] == 'Mon']['txn_week'], y = hex_2124_spend[hex_2124_spend['day'] == 'Mon']['Spend_Value'],
                         name = '21-24', visible = True))

multi_ddplot(which_fig = fig, df_list = [hex_0003_spend, hex_0306_spend, hex_0609_spend, hex_0912_spend, hex_1215_spend, hex_1518_spend, hex_1821_spend,hex_2124_spend],
             x_col = 'txn_week', menu_options_col = 'day', button_list = ['Spend_Value', 'Spend_Frequency'],
             buttonposition = [0.1, 0.6], labelposition = [0.03, 0.55], buttonheight = 1.15)

pretty_plot(fig, '<b>CAZ Transactions by Timeslot (Sum)</b>','Week','Transactions')
fig.show()

### **Daytime CAZ Footfall & Spend Comparison**

In [ ]:
hex_CAZ_visff = hex_tod_footfall[(hex_tod_footfall['area'] == 'CAZ') & (hex_tod_footfall['time_of_day'] == 'daytime')][['count_date', 'Visitor']]
hex_NCAZ_worff2 = hex_tod_footfall[(hex_tod_footfall['area'] == 'Non-CAZ') & (hex_tod_footfall['time_of_day'] == 'daytime')][['count_date', 'Worker']]
corr = pd.DataFrame(pd.date_range(start = "2022-05-01", end = "2023-12-31"), columns=['count_date'])
hex_CAZ_visff['count_date'] = pd.to_datetime(hex_CAZ_visff['count_date'])
hex_CAZ_worff2['count_date'] = pd.to_datetime(hex_CAZ_worff2['count_date'])
# hex_CAZ_resff['count_date'] = pd.to_datetime(hex_CAZ_resff['count_date'])
# hex_NCAZ_worff2['count_date'] = pd.to_datetime(hex_NCAZ_worff2['count_date'])
corr = corr.merge(right = hex_CAZ_visff, on = 'count_date', how = 'left')\
           .merge(right = hex_CAZ_worff2, on = 'count_date', how = 'left')
corr.columns = ['count_date', 'CAZ Visitor', 'CAZ Worker']

In [ ]:
hex_daily_spend = hex_spend[hex_spend['time_of_day'] == 'daytime'][['count_date', 'Spend_Value', 'Spend_Frequency']]
hex_daily_spend['count_date'] = pd.to_datetime(hex_daily_spend['count_date'])
corr = corr.merge(right = hex_daily_spend, on = 'count_date', how = 'left')
corr.columns = ['count_date', 'CAZ Visitor', 'CAZ Worker', 'CAZ Spend Value', 'CAZ Spend Frequency']
corr['day'] = corr['count_date'].dt.isocalendar().day
days = {1:'Mon', 2:'Tue', 3:'Wed', 4:'Thu', 5:'Fri', 6:'Sat', 7:'Sun'}
corr['day'] = corr['day'].map(days)
# corr.drop('count_date', axis = 1, inplace = True)
matrix = np.triu(corr[corr['CAZ Spend Value'].notnull()][['CAZ Visitor', 'CAZ Worker', 'CAZ Spend Value', 'CAZ Spend Frequency']].corr())
# using the upper triangle matrix as mask
fig = sns.heatmap(corr[corr['CAZ Worker'].notnull()][['CAZ Visitor', 'CAZ Worker', 'CAZ Spend Value', 'CAZ Spend Frequency']].corr(), annot=True, mask=matrix)
fig.set_title('Correlation Heatmap')

In [ ]:
week_colours = ['rgb(238,245,251)', 'rgb(226,238,248)', 'rgb(197,220,242)', 'rgb(168,203,235)', 'rgb(139,185,229)', colours[1], 'rgb(109,167,222)']
fig = px.scatter(corr[(corr['day'].isin(['Wed', 'Fri']))], y = 'CAZ Spend Value', x = 'CAZ Visitor', color_discrete_sequence = week_colours[4:6], color='day')
fig.add_shape(type = 'line', y0 = 420000, x0 = 500, y1 = 1080000, x1 = 3200, line = dict(color = "rgba(100,100,100,1)", dash = 'solid', width = 3), xref = 'x', yref = 'y')
pretty_plot(fig, '<b>Daytime CAZ Visitor vs Spend Value</b>', '', 'Total Transaction Value')
size_plot(fig, 600, 750)
fig.update_layout(font_color="rgb(0, 0, 0)", plot_bgcolor = 'rgba(255, 255, 255, 0)',
                  paper_bgcolor = 'rgba(255, 255, 255, 0)', xaxis = dict(showgrid = False, showticklabels = False), yaxis = dict(showgrid = False, showticklabels = False))
fig.show()
pio.write_image(fig, file = 'Daytime CAZ Visitor vs Spend Value.png', format = 'png', scale=6, width=500, height=600)

In [ ]:
# fig = px.scatter(corr, x = 'CAZ Visitor', y = 'CAZ Spend Frequency')
# size_plot(fig, 600, 750)
# fig.update_layout(title = dict(text = '<b>Correlation Between Daytime CAZ Visitor Footfall & Spend Frequency</b>', font = (dict(size=22))))
# fig.show()

In [ ]:
corr.head()

In [ ]:
fig = px.scatter(corr[(corr['day'].isin(['Wed', 'Fri']))], y = 'CAZ Spend Frequency', x = 'CAZ Worker', color_discrete_sequence = week_colours[4:6], color='day')
fig.add_shape(type = 'line', y0 = 600000, x0 = 500, y1 = 1400000, x1 = 2400, line = dict(color = "rgba(100,100,100,1)", dash = 'solid', width = 3), xref = 'x', yref = 'y')
pretty_plot(fig, '<b>Daytime CAZ Worker vs Spend Frequency</b>', '', 'Number of Transactions')
size_plot(fig, 600, 750)
fig.update_layout(font_color="rgb(0, 0, 0)", plot_bgcolor = 'rgba(255, 255, 255, 0)',
                  paper_bgcolor = 'rgba(255, 255, 255, 0)', xaxis = dict(showgrid = False, showticklabels = False), yaxis = dict(showgrid = False, showticklabels = False))
fig.show()
pio.write_image(fig, file = 'Daytime CAZ Worker vs Spend Frequency.png', format = 'png', scale=6, width=500, height=600)

In [ ]:
fig = px.scatter(corr[~(corr['day'].isin(['Sat', 'Sun']))], x = 'CAZ Spend Frequency', y = 'CAZ Spend Value', color_discrete_sequence = week_colours[1:], color='day')
fig.add_shape(type = 'line', x0 = 700000, y0 = 400000, x1 = 1350000, y1 = 1100000, line = dict(color = "rgba(100,100,100,1)", dash = 'solid', width = 3), xref = 'x', yref = 'y')
pretty_plot(fig, '<b>Daytime Spend Value vs Spend Frequency</b>', '', 'Total Transaction Value')
size_plot(fig, 600, 750)
fig.update_layout(font_color="rgb(0, 0, 0)", plot_bgcolor = 'rgba(255, 255, 255, 0)',
                  paper_bgcolor = 'rgba(255, 255, 255, 0)', xaxis = dict(showgrid = False, showticklabels = False, range = [400000, 1500000]), yaxis = dict(showgrid = False, showticklabels = False, range = [400000, 1200000]))
fig.show()
pio.write_image(fig, file = 'Daytime CAZ Spend Frequency vs Spend Value.png', format = 'png', scale=6, width=500, height=600)